In [2]:
from langchain_openai import AzureChatOpenAI

import os

llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [14]:
def function_1(input_1):
    query = f""" Your task is to provide only the topic based on the user query.
Only output te topic among: [Japan, Sports]. Don't include reasoning. 
Following the user query: {input_1}
"""
    response = llm.invoke(query)
    return response.content

def function_2(input_2):
    TOPIC_UPPER = input_2.upper()
    response = f"Here is the topic in UPPER case: {TOPIC_UPPER}"
    return response

In [15]:
from langgraph.graph import Graph

workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge("node_1", "node_2")

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [18]:
app.invoke('The weather in mexico is sunny')

'Here is the topic in UPPER case: SPORTS'

In [19]:
input = "The basket match was too long "

for output in app.stream(input):

    for key, value in output.items():
        print(f"output from node {key} ")
        print("----")
        print(value)

output from node node_1 
----
Sports
output from node node_2 
----
Here is the topic in UPPER case: SPORTS


In [23]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=api_base,
                api_key=api_key,
                azure_deployment=embeddings_model)

index_path = "../api/index/faiss_index"

    
vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()
#documents = retriever.invoke(query)
#result_string = "\n\n".join(doc.page_content for doc in documents)

    

In [50]:
retriever.invoke("gen ai in art")

[Document(id='fc8c5482-ab44-4f23-b63d-246243627280', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-08T00:43:38+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-08T00:43:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\p.a.rodriguez.canedo\\Documents\\Hackaton_2025\\api\\index_documents\\genai-art.pdf', 'total_pages': 23, 'page': 0, 'page_label': '1'}, page_content='Art and the science of generative AI: A deeper dive\nZiv Epstein1∗, Aaron Hertzmann2, Laura Herman3,4, Robert Mahari1,5,\nMorgan R. Frank 6, Matthew Groh1, Hope Schroeder1, Amy Smith7,\nMemo Akten8, Jessica Fjeld5, Hany Farid9, Neil Leach10,\nAlex “Sandy” Pentland1, Olga Russakovsky11\n1MIT Media Lab\n2Adobe Research\n3University of Oxford\n4Adobe, Inc.\n5Harvard Law School\n6University of Pittsburgh\n7Queen Mary University 

In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import retrieval_qa



In [49]:
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing import Annotated, Sequence
import operator

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
def function_1(state: State):
    messages = state['messages']
    question = messages[-1]

    query = f""" Your task is to provide only the topic based on the user query.
Only output the topic among: [GenAI For art, Gen ai for music]. Don't include reasoning. 
Following the user query: {question}
"""
    response = llm.invoke(query)

    state['messages'].append(response.content)
    return state

def function_2(state: State):
    messages = state['messages']
    question = messages[0]

    template = "Answer questions based only on the following context: {context} Question: {question}"
    
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    result = retrieval_chain.invoke(question)
    
    return result

In [28]:
workflow = Graph()

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge("agent", "tool")

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [30]:
inputs = {"messages": ["Tell me how to create art with generative AI using python"]}

print(app.invoke(inputs))

To create art with generative AI using Python, follow these steps based on the provided context:

1. **Install Necessary Software**: Ensure you have Python installed on your computer along with required libraries. You can install libraries like Matplotlib and NumPy using pip.

2. **Define the Art Piece**: Decide on the concept for your generative art piece. It may be a pixel-based colored design intended for a digital poster background.

3. **Set Up the Canvas**: Create a blank canvas using NumPy with specified width and height to serve as the base for your artwork.

4. **Manipulate the Canvas**: Use algorithms to generate the artwork. Define parameters such as a spiral pattern, and iterate through a range of points to calculate their positions using polar coordinates. Assign random colors to each point to create visual interest.

5. **Display the Art**: Utilize Matplotlib to visualize the generated art. Make sure to turn off the axes to ensure the focus is on the artwork itself.

6. *

In [32]:


for output in app.stream(inputs):

    for key, value in output.items():
        print(f"output from node {key} ")
        print("----")
        print(value)

output from node agent 
----
{'messages': ['Tell me how to create art with generative AI using python', 'GenAI For art', 'GenAI For art']}
output from node tool 
----
To create art with generative AI using Python, you can follow these steps:

1. **Install Python and Required Libraries**: Make sure Python is installed on your computer. You’ll also need to install necessary libraries such as `matplotlib` for plotting and `numpy` for numerical operations. You can install `matplotlib` using pip:
   ```
   pip install matplotlib
   ```

2. **Define Your Generative Art Piece**: Decide what type of generative art you want to create. For example, you could aim for a pixel-based colored concept suitable for a digital poster background.

3. **Set Up Your Coding Environment**: Open your preferred code editor or IDE, and start a new Python project. Import the required libraries:
   ```python
   import numpy as np
   import matplotlib.pyplot as plt
   ```

4. **Create a Blank Canvas**: Use NumPy to

In [34]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field

class TopicSelectionParser(BaseModel):
    Topic: str = Field(description="Selected topic")

parser = PydanticOutputParser(pydantic_object=TopicSelectionParser)

In [51]:
def function_1(state: State):
    print("Calling Agent...")
    messages = state['messages']
    question = messages[-1]

    template = """ Your task is to provide only the topic based on the user query.
Only output the topic among: [GenAI Art, GenAI Music, Not Related]. Don't include reasoning. 
Following the user query: {question}
{format_instructions}
"""
    prompt = PromptTemplate(
        template = template,
        input_variables=[question],
        partial_variables={"format_instructions" : parser.get_format_instructions()}
    )

    chain = prompt | llm | parser
    response = chain.invoke({"question": question, "format_instructions": parser.get_format_instructions()})


    print(response)
    #state['messages'].append(response.content)
    return {"messages": [response.Topic]}

def function_2(state: State):
    print("Calling RAG...")
    messages = state['messages']
    question = messages[0]

    template = "Answer questions based only on the following context: {context} Question: {question}"
    
    prompt = ChatPromptTemplate.from_template(template)

    retrieval_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    result = retrieval_chain.invoke(question)
    
    return {"messages": [result]}

def function_3(state: State):
    print("calling LLM...")

    messages = state['messages']
    question = messages[0]

    query = f"Answer the following question with your knowledge of the real world. Question: {question}"

    response = llm.invoke(query)
    return {"messages": [response.content]}


def router(state: State):
    print("Router...")
    messages = state['messages']
    last_message = messages[-1]
    print(f"last message: {last_message}")
    
    if 'GenAI Art' in last_message or 'GenAI Music' in last_message:
        return 'RAG'
    else:
        return 'LLM'



In [52]:
from langgraph.graph import StateGraph, END

graph = StateGraph(State)

graph.add_node("agent", function_1)
graph.add_node("RAG", function_2)
graph.add_node("LLM", function_3)

graph.set_entry_point("agent")

graph.add_conditional_edges(
    "agent",
    router
   # conditional_edge_mapping = {
   #     "RAG Call": "RAG",
   #     "LLM Call": "LLM"
   # }
)

graph.add_edge("RAG", END)
graph.add_edge("LLM", END)


app = graph.compile()

In [55]:
inputs = {"messages": ["Can Art be created with AI TOOLS ?"]}

print(app.invoke(inputs))

Calling Agent...
Topic='GenAI Art'
Router...
last message: GenAI Art
Calling RAG...
{'messages': ['Can Art be created with AI TOOLS ?', 'GenAI Art', 'Yes, art can be created with AI tools. Generative AI is described as a new class of tools that can produce high-quality artistic media across various forms, including visual arts, music, literature, and more. Additionally, generative art, which often involves algorithms and programming, creates artwork that can be entirely or partially generated autonomously, showcasing the collaboration between human creativity and AI capabilities.']}
